In [1]:
import cv2
import numpy as np
import cv2
import time
from IPython.display import clear_output

WIND_X = 480
WIND_Y = 360

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

def turn_to_RG (img):
    (h, w, d) = img.shape
    
    norm = np.zeros ((h, w), np.float)
    norm = img [:, :, 0].astype ('float') +\
           img [:, :, 1].astype ('float') +\
           img [:, :, 2].astype ('float')
    
    norm [norm == 0] = 5
    
    turned = np.zeros (img.shape, np.uint8)
    turned [:, :, 0] = ((img [:, :, 0].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 1] = ((img [:, :, 1].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 2] = ((img [:, :, 2].astype ('float')) / norm * 255).astype ('uint8')
    
    return turned

def obtain_color_ratio_mask (img, components, th, bl, use_rg = False, use_morph = False):
    sh = img [:, :, 0].shape
    
    rg = img
    
    if (use_rg == True):
        rg = turn_to_RG (img)
    
    smoothed = cv2.blur (rg, (bl, bl))
    
    needed = img.copy ()
    needed [:, :, 0] = np.full (sh, components [0])
    needed [:, :, 1] = np.full (sh, components [1])
    needed [:, :, 2] = np.full (sh, components [2])
    
    diff = cv2.absdiff (smoothed, needed)
    
    dif = diff [:, :, 0] + diff [:, :, 1] + diff [:, :, 2]
    
    ret, res_mask = cv2.threshold (dif, th, 255, cv2.THRESH_BINARY_INV)
    
    if (use_morph == True):
        res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_ERODE, np.ones ((int (bl), int (bl)), np.uint8))
        res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_CLOSE, np.ones ((int (bl), int (bl)), np.uint8))
    
    res = cv2.bitwise_and (img, img, mask = res_mask)
    
    return res, res_mask

def find_max_bounding_box (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_WIDTH] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_WIDTH]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    return (left, top), (left + width, top + height)

In [2]:
#cam = cv2.VideoCapture (1)
cam = cv2.VideoCapture ("inp.webm")

#cv2.namedWindow  ("field", cv2.WINDOW_NORMAL)
#cv2.resizeWindow ("field", (WIND_X, WIND_Y))

output_shape = (WIND_X * 3, WIND_Y * 2)

fourcc = cv2.VideoWriter_fourcc (*'XVID')
out    = cv2.VideoWriter ('output.avi', fourcc, 20.0, output_shape)

cv2.namedWindow  ("goal detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("goal detection", output_shape)

while (cam.isOpened ()):
    ret1, frame = cam.read ()
    
    if (ret1 != True):
        break
    
    frame_resized = cv2.resize (frame, (int (WIND_X), int (WIND_Y)))
    
    cv2.waitKey (1)
    
    goal1, g_mask1 = obtain_color_ratio_mask (frame_resized, (245, 245, 245), 40, 3, use_morph = True)    
    goal2, g_mask2 = obtain_color_ratio_mask (frame_resized, (85, 85, 85), 15, 1, use_rg = True, use_morph = True)    
    
    goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    goal_res = cv2.bitwise_and (frame_resized, frame_resized, mask = goal_mask_res)
    
    tl, br = find_max_bounding_box (goal_mask_res)
    
    goal_on_inp = cv2.rectangle (frame_resized.copy (), tl, br, (255, 0, 0), 5)
    
    goal_marked = cv2.addWeighted (frame_resized, 0.6, goal_on_inp, 0.4, 0)
    
    #output_image = np.concatenate ((frame_resized, goal_res), axis = 1)
    output_image = np.concatenate ((frame_resized, goal_marked), axis = 1)
    
    cv2.imshow ("goal detection", output_image)
    out.write (output_image)
    
    time.sleep  (0.01)
    
    if (cv2.waitKey (1) & 0xFF == ord ('q')):
        break

cam.release ()
out.release ()

cv2.waitKey           (1)
cv2.destroyAllWindows ()

In [2]:
get_available_cameras (10)

[0, 1]